# This Amazing Work is Done By :
# Eng.Ahmad Mohamd bozakly
# Eng.hamaz shnsho
# Eng.Marwan Kar ali 

In [1]:
pip install pysimplegui

Note: you may need to restart the kernel to use updated packages.


In [2]:
from experta import *
import PySimpleGUI as sg


class Answer(Fact):
    ident = Field(str)
    text = Field(str)


class Question(Fact):
    ident = Field(str)
    questionType = Field(str)
    valid = Field(list)
    text = Field(str)


class Ask(Fact):
    questionIdent = Field(str)


class DiagnosticExpertEngine(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.gui = None

    @DefFacts()
    def initialize_all_the_questions_here(self):
        yield Question(
            ident="R01",
            questionType="multi",
            valid=["yes", "no"],
            text="Do you feel tired quickly?"
        )
        yield Question(
            ident="R02",
            questionType="multi",
            valid=["yes", "no"],
            text="Does the chest feel painful?"
        )
        yield Question(
            ident="R03",
            questionType="multi",
            valid=["yes", "no"],
            text="Do you feel anxious?"
        )
        yield Question(
            ident="R04",
            questionType="multi",
            valid=["yes", "no"],
            text="Does your back hurt?"
        )
        yield Question(
            ident="R05",
            questionType="multi",
            valid=["yes", "no"],
            text="Does the heart beat faster?"
        )
        yield Question(
            ident="R06",
            questionType="multi",
            valid=["yes", "no"],
            text="Do you have sleep disturbances?"
        )
        yield Question(
            ident="R07",
            questionType="multi",
            valid=["yes", "no"],
            text="Are your knees, elbows, wrists, hands and feet painful?"
        )
        yield Question(
            ident="R08",
            questionType="multi",
            valid=["yes", "no"],
            text="Does the chest feel painful?"
        )
        yield Question(
            ident="R10",
            questionType="multi",
            valid=["yes", "no"],
            text="Does your breath feel short?"
        )
        yield Question(
            ident="R11",
            questionType="multi",
            valid=["yes", "no"],
            text="Are you easy to forget?"
        )
        yield Question(
            ident="R12",
            questionType="multi",
            valid=["yes", "no"],
            text="Does the head often hurt?"
        )
        yield Question(
            ident="R13",
            questionType="multi",
            valid=["yes", "no"],
            text="Is appetite reduced?"
        )
        yield Question(
            ident="R14",
            questionType="multi",
            valid=["yes", "no"],
            text="Does the chest hurt or feel oppressed?"
        )

    def validate_answer(self, answer, questionType, valid):
        if answer in valid:
            return True
        return False

    def ask_user(self, question, questionType, valid=None):
        layout = [
            [sg.Text(question)],
            [sg.Text("Valid answers are: " + ", ".join(valid))],
            [sg.InputText(key="ANSWER")],
            [sg.Button("Submit")]
        ]

        window = sg.Window("Question", layout)

        while True:
            event, values = window.read()
            if event == sg.WINDOW_CLOSED:
                exit()
            elif event == "Submit":
                answer = values["ANSWER"]
                if self.validate_answer(answer, questionType, valid):
                    window.close()
                    return answer
                else:
                    sg.popup("Invalid answer, please try again.")

    def recommend_action(self, action):
        sg.popup("Recommendation", "I recommend that you " + action + "\n\n")

    def success(self):
        sg.popup("Result", "Your health is good, no further actions should be taken.")

    def failure(self):
        sg.popup("Result", "A critical issue has been detected with your health.")
        self.recommend_action('consult an expert Doctor')

    @Rule(
        Question(ident=MATCH.id, text=MATCH.text, valid=MATCH.valid, questionType=MATCH.questionType),
        NOT(Answer(ident=MATCH.id)),
        AS.ask << Ask(questionIdent=MATCH.id)
    )
    def ask_question_by_ident(self, ask, id, text, valid, questionType):
        self.retract(ask)
        answer = self.ask_user(text, questionType, valid)
        self.declare(Answer(ident=id, text=answer))

    @Rule(
        Answer(ident="R01", text="yes"),
        NOT(Answer(ident="R02"))
    )
    def ask_chest_pain(self):
        self.declare(Ask(questionIdent="R02"))

    @Rule(
        Answer(ident="R01", text="no"),
        NOT(Answer(ident="R10"))
    )
    def ask_short_breath(self):
        self.declare(Ask(questionIdent="R10"))

    @Rule(
        Answer(ident="R02", text="yes"),
        NOT(Answer(ident="R04"))
    )
    def ask_back_pain(self):
        self.declare(Ask(questionIdent="R04"))

    @Rule(
        Answer(ident="R02", text="no"),
        NOT(Answer(ident="R03"))
    )
    def ask_anxious(self):
        self.declare(Ask(questionIdent="R03"))

    @Rule(
        Answer(ident="R04", text="yes"),
        NOT(Answer(ident="R07"))
    )
    def ask_joint_pain(self):
        self.declare(Ask(questionIdent="R07"))

    @Rule(
        Answer(ident="R04", text="no"),
        NOT(Answer(ident="R06"))
    )
    def ask_sleep_disturbances(self):
        self.declare(Ask(questionIdent="R06"))

    @Rule(
        Answer(ident="R07", text="yes")
    )
    def heart_issue_swelling(self):
        sg.popup("Result", "Swelling in legs or ankles can be linked to heart issues. Seek evaluation.")
        self.halt()

    @Rule(
        Answer(ident="R07", text="no"),
        Answer(ident="R04", text="yes"),
        NOT(Answer(ident="R13"))
    )
    def ask_appetite_joint_pain(self):
        self.declare(Ask(questionIdent="R13"))

    @Rule(
        Answer(ident="R06", text="no"),
        Answer(ident="R04", text="no"),
        Answer(ident="R02", text="yes")
    )
    def chest_hurt(self):
        self.declare(Ask(questionIdent="R14"))

    @Rule(
        Answer(ident="R06", text="yes")
    )
    def forget_easy(self):
        self.declare(Ask(questionIdent="R11"))

    @Rule(
        Answer(ident="R10", text="yes"),
        NOT(Answer(ident="R05"))
    )
    def ask_heart_rate(self):
        self.declare(Ask(questionIdent="R05"))

    @Rule(
        Answer(ident="R10", text="no"),
        NOT(Answer(ident="R12"))
    )
    def ask_headache(self):
        self.declare(Ask(questionIdent="R12"))

    @Rule(
        Answer(ident="R05", text="yes")
    )
    def ask_anxious_heart_rate(self):
        sg.popup("Result", "No specific heart disease symptoms detected. However, if concerned, seek medical advice.")
        self.halt()

    @Rule(
        Answer(ident="R03", text="yes")
    )
    def chest_hurt(self):
        self.declare(Ask(questionIdent="R14"))

    @Rule(
        Answer(ident="R03", text="no")
    )
    def chest_hurt(self):
        self.declare(Ask(questionIdent="R13"))

    @Rule(
        Answer(ident="R03", text="no"),
        Answer(ident="R05", text="yes")
    )
    def no_anxiety_fast_heart_rate(self):
        sg.popup("Result", "Fast heart rate can be a symptom of heart issues. Consult a cardiologist.")
        self.halt()

    @Rule(
        Answer(ident="R03", text="no"),
        Answer(ident="R05", text="no"),
        NOT(Answer(ident="R13"))
    )
    def ask_appetite(self):
        self.declare(Ask(questionIdent="R13"))

    @Rule(
        Answer(ident="R13", text="yes")
    )
    def nausea_vomiting(self):
        sg.popup("Result", "Nausea or vomiting may be related to heart conditions. Seek medical attention.")
        self.halt()

    @Rule(
        Answer(ident="R13", text="no")
    )
    def no_specific_heart_symptoms(self):
        sg.popup("Result", "No specific heart disease symptoms detected. However, if concerned, seek medical advice.")
        self.halt()

    @Rule(
        Answer(ident="R12", text="no")
    )
    def myocardial_infarction_headache(self):
        sg.popup("Result", "No specific heart disease symptoms detected. However, if concerned, seek medical advice.")
        self.halt()

    @Rule(
        Answer(ident="R12", text="yes"),
        NOT(Answer(ident="R11"))
    )
    def ask_forgetful(self):
        self.declare(Ask(questionIdent="R11"))

    @Rule(
        Answer(ident="R11", text="no")
    )
    def myocardial_infarction_forgetful(self):
        sg.popup("Result", "Myocardial Infarction (Heart attack). Severe chest pain. Seek immediate medical attention. Life-threatening.")
        self.halt()

    @Rule(
        Answer(ident="R11", text="yes")
    )
    def no_heart_issue(self):
        sg.popup("Result", "No specific heart disease symptoms detected. However, if concerned, seek medical advice.")
        self.halt()

    @Rule(
        Answer(ident="R14", text="yes")
    )
    def possible_heart_disease(self):
        sg.popup("Result", "Possible heart disease. Chest pain is a significant symptom. Consult a cardiologist.")
        self.halt()

    @Rule(
        Answer(ident="R14", text="no")
    )
    def heart_failure(self):
        sg.popup("Result", "Heart failure. Heart's inability to pump blood effectively. Causes breathing difficulties.")
        self.halt()


def main():
    engine = DiagnosticExpertEngine()
    engine.reset()
    engine.declare(Ask(questionIdent="R01"))
    engine.run()


if __name__ == "__main__":
    main()
